In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row

Install findspark to make available spark from jupyter-python

In [2]:
#!conda insstall openjdk
#!conda install findspark
#install hadoop:
#https://github.com/ruslanmv/How-to-install-Hadoop-on-Windows?tab=readme-ov-file

In [3]:
import findspark
findspark.init()
findspark.find()

'c:\\soft\\miniconda3\\envs\\iceberg_env\\lib\\site-packages\\pyspark'

In [4]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.4.1
      /_/
                        
Using Scala version 2.12.17, OpenJDK 64-Bit Server VM, 11.0.13
Branch HEAD
Compiled by user centos on 2023-06-19T23:01:01Z
Revision 6b1ff22dde1ead51cbf370be6e48a802daae58b6
Url https://github.com/apache/spark
Type --help for more information.


In [5]:
#spark.stop()

In [6]:
catalog_nm = "flights_catalog"
layer_nm = "silver" #options: "silver", "gold"
warehouse_path = f"file:/C:/ws/github/iceberg-lakehouse/warehouse_flights/{layer_nm}"

conf = (
    pyspark.SparkConf()
    .setAppName("iceberg_flights_app")
    #packages
    .set("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.5.2") #spark: 3.4, scala:2.12 iceberg:1.5.2
    #SQL extensions
    .set("spark.sql.extensions","org.apache.iceberg.spark.extensions:IcebergSparkSessionExtensions")
    #Configuring catalog
    .set(f"spark.sql.catalog.{catalog_nm}","org.apache.iceberg.spark.SparkCatalog")
    .set(f"spark.sql.catalog.{catalog_nm}.type","hadoop")
    .set(f"spark.sql.catalog.{catalog_nm}.warehouse",warehouse_path)
)

#Start SparkSession
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

Spark Running


In [7]:
spark

In [8]:
sales_table_nm = "sales"
#silver_sales_loc = f"{silver_bucket}_{sales_table_nm}"
#print(silver_sales_loc)

In [9]:
spark.sql(
    f"""
    CREATE TABLE IF NOT EXISTS {catalog_nm}.{sales_table_nm} (
        transaction_id STRING NOT NULL,
        customer_id STRING NOT NULL,
        transaction_timestamp STRING NOT NULL,
        price STRING NOT NULL,
        currency STRING NOT NULL,
        flight_id STRING NOT NULL,
        booking_reference STRING COMMENT 'May be Null if reserved with a Travel Agency',
        year_ptt INT NOT NULL,
        month_ptt INT NOT NULL,
        day_ptt INT NOT NULL
    )
    USING iceberg
    PARTITIONED BY (year_ptt, month_ptt, day_ptt)
    --LOCATION sales_table_nm;
    """
).show()

++
||
++
++



In [10]:
sales_df = spark.sql(
    f"""
    SELECT * 
    FROM {catalog_nm}.{sales_table_nm}
    LIMIT 10;
    """
)
sales_df.show()

+--------------+-----------+---------------------+-----+--------+---------+-----------------+--------+---------+-------+
|transaction_id|customer_id|transaction_timestamp|price|currency|flight_id|booking_reference|year_ptt|month_ptt|day_ptt|
+--------------+-----------+---------------------+-----+--------+---------+-----------------+--------+---------+-------+
+--------------+-----------+---------------------+-----+--------+---------+-----------------+--------+---------+-------+



In [11]:
spark.sql(
    f"""
    SELECT COUNT(*) 
    FROM {catalog_nm}.{sales_table_nm};
    """
).show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [12]:
sales_df.printSchema()

root
 |-- transaction_id: string (nullable = false)
 |-- customer_id: string (nullable = false)
 |-- transaction_timestamp: string (nullable = false)
 |-- price: string (nullable = false)
 |-- currency: string (nullable = false)
 |-- flight_id: string (nullable = false)
 |-- booking_reference: string (nullable = true)
 |-- year_ptt: integer (nullable = false)
 |-- month_ptt: integer (nullable = false)
 |-- day_ptt: integer (nullable = false)

